In [ ]:
import plotly.graph_objects as go
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
colors = {
    'ARI':["#97233F","#000000","#FFB612"],
    'ATL':["#A71930","#000000","#A5ACAF"],
    'BAL':["#241773","#000000"],
    'BUF':["#00338D","#C60C30"],
    'CAR':["#0085CA","#101820","#BFC0BF"],
    'CHI':["#0B162A","#C83803"],
    'CIN':["#FB4F14","#000000"],
    'CLE':["#311D00","#FF3C00"],
    'DAL':["#003594","#041E42","#869397"],
    'DEN':["#FB4F14","#002244"],
    'DET':["#0076B6","#B0B7BC","#000000"],
    'GB' :["#203731","#FFB612"],
    'HOU':["#03202F","#A71930"],
    'IND':["#002C5F","#A2AAAD"],
    'JAX':["#101820","#D7A22A","#9F792C"],
    'KC' :["#E31837","#FFB81C"],
    'LA' :["#003594","#FFA300","#FF8200"],
    'LAC':["#0080C6","#FFC20E","#FFFFFF"],
    'LV' :["#000000","#A5ACAF"],
    'MIA':["#008E97","#FC4C02","#005778"],
    'MIN':["#4F2683","#FFC62F"],
    'NE' :["#002244","#C60C30","#B0B7BC"],
    'NO' :["#101820","#D3BC8D"],
    'NYG':["#0B2265","#A71930","#A5ACAF"],
    'NYJ':["#125740","#000000","#FFFFFF"],
    'PHI':["#004C54","#A5ACAF","#ACC0C6"],
    'PIT':["#FFB612","#101820"],
    'SEA':["#002244","#69BE28","#A5ACAF"],
    'SF' :["#AA0000","#B3995D"],
    'TB' :["#D50A0A","#FF7900","#0A0A08"],
    'TEN':["#0C2340","#4B92DB","#C8102E"],
    'WAS':["#5A1414","#FFB612"],
    'football':["#CBB67C","#663831"]
}

In [ ]:
projectRoot = Path.cwd()
trackingDf = pd.read_csv(f'{projectRoot}/data/train/input_2023_w01.csv')
plays = pd.read_csv(f'{projectRoot}/data/supplementary_data.csv')

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# =====================================
# ROLE COLORS + MARKERS
# =====================================
def role_color(role):
    if pd.isna(role): return 'gray'
    if 'target' in str(role).lower(): return 'lime'
    if 'passer' in str(role).lower(): return 'orange'
    if 'defensive' in str(role).lower(): return '#2E86C1'
    if 'route' in str(role).lower(): return '#E74C3C'
    if 'ball' in str(role).lower(): return 'gold'
    return 'white'

# =====================================
# CONFIG
# =====================================
projectRoot = Path.cwd()

base_path = f"{projectRoot}/cleanedData/train"
week_file = "oriented_2023_w01.csv"  # ← you can change to any week
weekAfter_file = "output_oriented_2023_w01.csv"  # ← you can change to any week
save_dir = "frames_enhanced"
os.makedirs(save_dir, exist_ok=True)

# =====================================
# LOAD A SAMPLE PLAY
# =====================================
df = pd.read_csv(f"{base_path}/{week_file}")
df_after = pd.read_csv(f"{base_path}/{weekAfter_file}")
print("Data shape:", df.shape)
sample_play = [2023090700,194]
data = df[(df['game_id']==sample_play[0]) & (df['play_id']==sample_play[1])].copy()
max_frame = data['frame_id'].max()

players = data['nfl_id'].unique()
roleColorDict = {}

for i, pl in enumerate(players):
    role = data[(df['nfl_id']==pl)].iloc[0]['player_role']
    print(role)
    roleColorDict[pl]= role_color(role)

data_after = df_after[(df_after['game_id']==sample_play[0]) & (df_after['play_id']==sample_play[1])].copy()
data_after['frame_id'] = data_after['frame_id'] + max_frame
data = pd.concat([data,data_after])

frame_ids = sorted(data['frame_id'].unique())
print(f"Animating Game {sample_play[0]}, Play {sample_play[1]} with {len(frame_ids)} frames.")

# =====================================
# DRAW FOOTBALL FIELD
# =====================================
def draw_field(ax):
    ax.set_facecolor('#1E5128')
    ax.plot([0,120],[0,0],color='white')
    ax.plot([0,120],[53.3,53.3],color='white')
    for x in range(10,120,10):
        ax.plot([x,x],[0,53.3],color='white',linestyle='--',linewidth=0.8)
    ax.axvspan(0,10,facecolor='blue',alpha=0.15)
    ax.axvspan(110,120,facecolor='red',alpha=0.15)
    ax.set_xlim(0,120)
    ax.set_ylim(0,53.3)
    ax.axis('off')

def team_marker(side):
    return 'o' if side == 'home' else 's'

# =====================================
# MOTION TRAILS STORAGE
# =====================================
trail_length = 5
player_trails = {}

# =====================================
# GENERATE FRAMES
# =====================================
for i, frame_id in enumerate(frame_ids):
    frame = data[data['frame_id']==frame_id]
    fig, ax = plt.subplots(figsize=(16,8))
    draw_field(ax)

    for _, row in frame.iterrows():
        pid = row['nfl_id']
        player_trails.setdefault(pid, []).append((row['x'], row['y']))
        if len(player_trails[pid]) > trail_length:
            player_trails[pid].pop(0)

        # Draw trails
        trail = np.array(player_trails[pid])
        ax.plot(trail[:,0], trail[:,1], color=roleColorDict[row['nfl_id']], alpha=0.4, linewidth=2)

        # Draw player
        ax.scatter(row['x'], row['y'],
                   c=roleColorDict[row['nfl_id']],
                   s=150,
                   edgecolor='black',
                   marker=team_marker(row['player_side']),
                   zorder=3)

        # Add direction arrow (velocity vector)
        dx = np.cos(np.deg2rad(row['dir'])) * (row['s'] / 3)
        dy = np.sin(np.deg2rad(row['dir'])) * (row['s'] / 3)
        ax.arrow(row['x'], row['y'], dx, dy, color='white', head_width=0.6, alpha=0.6, zorder=2)

        # Label key players
        if "target" in str(row['player_role']).lower() or "passer" in str(row['player_role']).lower():
            ax.text(row['x'], row['y']+1.5, row['player_name'].split()[0],
                    color='white', fontsize=7, ha='center', weight='bold')

    # Ball landing marker
    if not pd.isna(frame.iloc[0]['ball_land_x']):
        ax.scatter(frame.iloc[0]['ball_land_x'], frame.iloc[0]['ball_land_y'],
                   color='yellow', s=300, marker='*', edgecolor='black', linewidth=1.5, zorder=4)

    title = f"Game {sample_play[0]} | Play {sample_play[1]} | Frame {frame_id}"
    ax.set_title(title, fontsize=14, color='white', pad=10)
    plt.tight_layout()
    plt.savefig(f"{save_dir}/frame_{i:04d}.png", dpi=120, bbox_inches='tight', facecolor='#1E5128')
    plt.close()

    if (i+1) % 10 == 0:
        print(f"Processed {i+1}/{len(frame_ids)} frames")

# =====================================
# CREATE GIF
# =====================================
print("Creating enhanced animation...")
frames = [Image.open(f"{save_dir}/frame_{i:04d}.png") for i in range(len(frame_ids)) if os.path.exists(f"{save_dir}/frame_{i:04d}.png")]
frames[0].save('nfl_play_enhanced.gif', save_all=True, append_images=frames[1:], duration=120, loop=0)
print("✅ Enhanced animation created: nfl_play_enhanced.gif")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/luisbapst/Code/BDB26/cleanedData/train/output_orientated_2023_w01.csv'

In [37]:
from IPython.display import HTML
HTML('<img src="nfl_play_enhanced.gif" width="1000">')